# Identidad

Sea la entidad Empleado con atributos Nombre, Apellido, RFC, CURP, NSS y Nacimiento. Como RFC y CURP tienen valores únicos, son nuestras llaves candidatas. Elijamos una de ellas como llave primaria, digamos RFC.

In [7]:
--Para dejar en blanco la base de datos PRUEBA ejecute este bloque
--Ejecute este bloque(colapsado) para usarla 
USE master
IF EXISTS 
   (
     SELECT name FROM master.dbo.sysdatabases 
     WHERE name = 'Pruebas'
    )
    DROP DATABASE Pruebas
GO

CREATE DATABASE Pruebas
GO

use Pruebas

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:05.263

## Paso1 ##
Cree la tabla `Empleados` sin llave primaria

In [8]:
CREATE TABLE Empleados
(
    Nombre nvarchar(30), 
    Apellido nvarchar(30), 
    RFC CHAR(13), 
    CURP CHAR(18), 
    NSS CHAR(20),
    Nacimiento DATE
)

Commands completed successfully.

Total execution time: 00:00:00.163

¿Que sucede si intento agregar dos veces el mismo empleado? (Ejecute dos veces o mas el siguiente bloque)

**Respuesta:** Se van a agregar aunque haya redundancia ya que la tabla no contiene una llave primaria.


In [10]:
--Agregamos un empleado
INSERT into Empleados (Nombre, Apellido, RFC, CURP, NSS, Nacimiento)
    VALUES ('Juan', 'Salinas García', 'SAGJ680701HYB', 'SAGJ6807010000000', '1234567', '19680701' )
SELECT * from Empleados

(1 row affected)

(2 rows affected)

Total execution time: 00:00:00.078

Nombre,Apellido,RFC,CURP,NSS,Nacimiento
Juan,Salinas García,SAGJ680701HYB,SAGJ6807010000000,1234567,1968-07-01
Juan,Salinas García,SAGJ680701HYB,SAGJ6807010000000,1234567,1968-07-01


**¡ATENCION!**

Elimine todas las filas de la tabla antes de proseguir

In [11]:
DELETE Empleados
SELECT * FROM Empleados

(2 rows affected)

(0 rows affected)

Total execution time: 00:00:00.197

Nombre,Apellido,RFC,CURP,NSS,Nacimiento


## Paso 2 ##
Ahora agreguemos una restricción de llave primaria con la columna `RFC`. Recuerde que una llave primaria no debe permitir nulos, así que antes debemos añadir también la restriccion `NOT NULL` a la columna.

In [16]:
ALTER TABLE Empleados
    ALTER COLUMN RFC CHAR(13) NOT NULL
 GO
ALTER TABLE Empleados
    ADD CONSTRAINT PK_RFC PRIMARY KEY (RFC)




Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.997

Intente agregar ahora dos veces al mismo empleado ¿Que sucede?
**Respuesta:**

No permite la inserción debido a que hay una violación en la llave primaria. Por lo tanto, no es posible añadir llaves duplicadas en la tabla.

In [18]:
INSERT into Empleados (Nombre, Apellido, RFC, CURP, NSS, Nacimiento)
    VALUES ('Juan', 'Salinas García', 'SAGJ680701HYB', 'SAGJ6807010000000', '1234567', '19680701' )
SELECT * from Empleados

: Msg 2627, Level 14, State 1, Line 1
Violation of PRIMARY KEY constraint 'PK_RFC'. Cannot insert duplicate key in object 'dbo.Empleados'. The duplicate key value is (SAGJ680701HYB).

The statement has been terminated.

(1 row affected)

Total execution time: 00:00:00.206

Nombre,Apellido,RFC,CURP,NSS,Nacimiento
Juan,Salinas García,SAGJ680701HYB,SAGJ6807010000000,1234567,1968-07-01


## Paso 3##
Para mostrar los indices de una tabla usamos `sp_helpindex [ @objname = ] 'name' `

In [22]:
--Muestre los indices de la tabla Empleados
sp_helpindex Empleados


Commands completed successfully.

Total execution time: 00:00:01.765

index_name,index_description,index_keys
PK_RFC,"clustered, unique, primary key located on PRIMARY",RFC


## Paso 4 #

Supongamos que cambiamos de idea y queremos que la llave primaria sea `CURP`. Para eso tenemos que eliminar la propiedad `PRIMARY KEY` de `RFC` con `ALTER TABLE ... DROP CONSTRAINT ` y luego ponerla en `CURP`

In [24]:
ALTER TABLE Empleados DROP CONSTRAINT PK_RFC
GO 
ALTER TABLE Empleados ALTER COLUMN CURP CHAR(18) NOT NULL
GO
ALTER TABLE Empleados ADD CONSTRAINT PK_CURP PRIMARY KEY(CURP)


: Msg 3728, Level 16, State 1, Line 1
'PK_RFC' is not a constraint.

: Msg 3727, Level 16, State 0, Line 1
Could not drop constraint. See previous errors.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.667

¿Se puede agregar ahora un empleado con CURP unico pero RFC repetido? Respuesta: Si es posible, ya que la unica restriccion que hay es en el CURP.

In [25]:
INSERT INTO Empleados (Nombre, Apellido, RFC, CURP, NSS, Nacimiento) VALUES ('Daniel','Campos','CAMF981214CT8','CAMF981214HSLMDR06',1234567,'19981214')

INSERT INTO Empleados (Nombre, Apellido, RFC, CURP, NSS, Nacimiento) VALUES ('Daniel','Campos','CAMF981214CT8','CAMF981214HSLMDR10',1234567,'19981214')

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.042

In [26]:
SELECT * FROM Empleados

(3 rows affected)

Total execution time: 00:00:00.189

Nombre,Apellido,RFC,CURP,NSS,Nacimiento
Daniel,Campos,CAMF981214CT8,CAMF981214HSLMDR06,1234567,1998-12-14
Daniel,Campos,CAMF981214CT8,CAMF981214HSLMDR10,1234567,1998-12-14
Juan,Salinas García,SAGJ680701HYB,SAGJ6807010000000,1234567,1968-07-01


¿Como lo podemos evitar? 

**Respuesta**

Modificando la columna RFC como UNIQUE

# Unique #

Agreguemos entonces una restricción `UNIQUE` a la tabla `Empleados` sobre la columna `RFC`


In [29]:
DELETE Empleados

(3 rows affected)

Total execution time: 00:00:00.129

In [30]:
ALTER TABLE Empleados
    ADD CONSTRAINT Unique_RFC UNIQUE(RFC)



Commands completed successfully.

Total execution time: 00:00:00.038

Intente ahora agregar un RFC repetido

In [33]:
INSERT INTO Empleados (Nombre, Apellido, RFC, CURP, NSS, Nacimiento) VALUES ('Daniel','Campos','CAMF981214CT8','CAMF981214HSLMDR06',1234567,'19981214')
INSERT INTO Empleados (Nombre, Apellido, RFC, CURP, NSS, Nacimiento) VALUES ('Daniel','Campos','CAMF981214CT8','CAMF981214HSLMDR10',1234567,'19981214')


(1 row affected)

: Msg 2627, Level 14, State 1, Line 2
Violation of UNIQUE KEY constraint 'Unique_RFC'. Cannot insert duplicate key in object 'dbo.Empleados'. The duplicate key value is (CAMF981214CT8).

The statement has been terminated.

Total execution time: 00:00:00.031

In [34]:
SELECT * FROM Empleados

(1 row affected)

Total execution time: 00:00:00.074

Nombre,Apellido,RFC,CURP,NSS,Nacimiento
Daniel,Campos,CAMF981214CT8,CAMF981214HSLMDR06,1234567,1998-12-14


Revise los indices de la tabla Empleados

In [35]:
sp_helpindex Empleados

Commands completed successfully.

Total execution time: 00:00:00.548

index_name,index_description,index_keys
PK_CURP,"clustered, unique, primary key located on PRIMARY",CURP
Unique_RFC,"nonclustered, unique, unique key located on PRIMARY",RFC
